In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
model = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt','res10_300x300_ssd_iter_140000.caffemodel')
cap = cv2.VideoCapture(0)
while (True):
    success, img = cap.read()
    base_img = img.copy()
    original_size = img.shape
    tartget_size = (300,300)
    img = cv2.resize(img,tartget_size)
    aspect_ratio_y = original_size[0] / tartget_size [0] #help us in to apply the work on base_img
    aspect_ratio_x = original_size[1] / tartget_size[1]
    imgblob= cv2.dnn.blobFromImage(img) #prepare img and returen 4-dim (1,3,300,300)
    model.setInput(imgblob)
    detections = model.forward()
    detections_df = pd.DataFrame(detections[0][0], columns = ["img_id", "label", "confidence", "left", "top", "right", "bottom"])
    detections_df = detections_df.query('confidence > 0.9 and label == 1.0') #label of 1 means face\ 0:means background
    for i,instance in detections_df.iterrows():
        confidence = str(round(detections_df['confidence'][i]*100)) + "%"
        right = int(instance['right'] * 300) # multiply by 300 to see the results on the base_img remeber because we resize the img
        left = int(instance['left'] * 300)
        top = int(instance['top'] * 300)
        bottom = int(instance ['bottom'] * 300)
        detect_face = base_img[int(top *aspect_ratio_y): int(bottom*aspect_ratio_y), int(left*aspect_ratio_x): int(right*aspect_ratio_x)]
        if detect_face.shape[0] > 0 and detect_face.shape[1] > 0:
            cv2.putText(base_img, confidence , (int(left*aspect_ratio_x), int(top*aspect_ratio_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            cv2.rectangle(base_img, (int(left*aspect_ratio_x), int(top*aspect_ratio_y)), (int(right*aspect_ratio_x), int(bottom*aspect_ratio_y)), (255, 0, 0), 4) #draw rectangle to main image
    cv2.imshow('frame', base_img) #show window of your camera
    if cv2.waitKey(1) & 0xFF == ord('q'): #press q To exit
        break
    
    
cap.release()    
cv2.destroyAllWindows()
